# Data Loader Circuits

<div class="alert alert-block alert-warning">
    Note: The following notebook uses features of Forge (data loaders) which are not available to Developer-level (free) accounts.
</div>

Many quantum algorithms require the conversion of classical data into quantum states. QC Ware has developed carefully optimized NISQ circuits that perform this translation.

This notebook explains how to generate these circuits for your own data sets.

## Basic Usage

In [1]:
import numpy as np
from qcware.forge import qio
from qcware import forge

#### Generate a random vector

In [2]:
data = np.random.rand(4)-.5
normalized_data = data / np.linalg.norm(data)

print('Generated data:')
print(normalized_data)

Generated data:
[-0.64553254  0.31298566  0.01409863 -0.69651198]


#### Create the loader circuit

In [3]:
data_loader = qio.loader(normalized_data)

print(data_loader)

T  : |0|1|2|3|4|5|

q0 : -X-B-@-------
        | |       
q1 : ---S-|---@---
          |   |   
q2 : -----X-B-X-B-
            |   | 
q3 : -------S---S-
                  
T  : |0|1|2|3|4|5|



#### Find relevant indices
`qio.loader` can also tell us which indices of the statevector will contain our data.

In [4]:
data_loader, statevector_indices = qio.loader(
    normalized_data,
    return_statevector_indices=True
)
print("Statevector indices corresponding to our data elements:", statevector_indices)

# Some formatting
kets = ["|{:04b}>".format(i) for i in statevector_indices]
print("States corresponding to our data elements:", " ".join(kets))


Statevector indices corresponding to our data elements: [10  9  6  5]
States corresponding to our data elements: |1010> |1001> |0110> |0101>


#### Run the circuit on a simulator
We can look at how the data is stored after running the circuit. You should be able to find the data in the quantum statevector.

In [5]:
import quasar
statevector = quasar.QuasarSimulatorBackend().run_statevector(data_loader)

print(statevector)
print("\nEach element of our original data below should be somewhere in the above statevector!")
print("Original data:", normalized_data)

[ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.        +0.j -0.69651198+0.j  0.01409863+0.j  0.        +0.j
  0.        +0.j  0.31298566+0.j -0.64553254+0.j  0.        +0.j
  0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]

Each element of our original data below should be somewhere in the above statevector!
Original data: [-0.64553254  0.31298566  0.01409863 -0.69651198]


#### Reconstruct our original data from the simulation statevector
We can do this with the `statevector_indices` we recieved.

In [6]:
print(statevector[statevector_indices])

[-0.64553254+0.j  0.31298566+0.j  0.01409863+0.j -0.69651198+0.j]


That looks the same as above!

## Loader Type Selection

There are two types of loaders that we have designed: parallel loaders (which have less depth but more qubits) and optimized loaders (which have fewer qubits with greater depth).

In [7]:
data = np.random.rand(5)
normalized_data = data / np.linalg.norm(data)

parallel_loader = qio.loader(normalized_data, mode='parallel')
optimized_loader = qio.loader(normalized_data, mode='optimized')

In [8]:
print('Parallel Loader (more qubits, less depth):\n')
print(parallel_loader, '\n\n')

print('Optimized Loader (fewer qubits, greater depth):\n')
print(optimized_loader)

Parallel Loader (more qubits, less depth):

T  : |0|1|2|3|

q0 : -X-B-B-B-
        | | | 
q1 : ---|-|-S-
        | |   
q2 : ---|-S-B-
        |   | 
q3 : ---|---S-
        |     
q4 : ---S-----
              
T  : |0|1|2|3|
 


Optimized Loader (fewer qubits, greater depth):

T  : |0|1|2|3|4|5|6|7|8|

q0 : -X-B-@-------------
        | |             
q1 : ---S-|-------@-----
          |       |     
q2 : -----X-B-B-B-X-B-B-
            | | |   | | 
q3 : -------|-S-|---|-S-
            |   |   |   
q4 : -------S---S---S---
                        
T  : |0|1|2|3|4|5|6|7|8|



## Running Loaders on Real Hardware (Ion Q)

For a realistic application, a user would first use a loader and then add additional gates to implement a quantum algorithm. However, for simplicity we will just demonstrate running and measuring a loader circuit by itself.

In [9]:
# Generate some data
data = np.random.rand(4)
normalized_data = data / np.linalg.norm(data)

# Make a loader circuit
loader = qio.loader(normalized_data, mode='optimized')

# Get an Ion Q backend 
ion_q_backend = forge.circuits.QuasarBackend('awsbraket/ionq')

# Uncomment this line to perform hardware run:
# result = ion_q_backend.run_measurement(circuit=loader, nmeasurement=64)

Note that the result of a physical hardware run may be delayed because of hardware availability. You can retrieve the result using the API tab on Forge when the computation has been completed.